In [1]:
import sys
sys.path.append('../')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import argparse
import gc
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob
from torchvision import transforms
from PIL import Image
import json
import pickle
import numpy as np
import imageio


In [ ]:
!unzip dataset_release.zip

## data_util

In [3]:
face_class = ['background', 'head', 'head***cheek', 'head***chin', 'head***ear', 'head***ear***helix',
              'head***ear***lobule', 'head***eye***botton lid', 'head***eye***eyelashes', 'head***eye***iris',
              'head***eye***pupil', 'head***eye***sclera', 'head***eye***tear duct', 'head***eye***top lid',
              'head***eyebrow', 'head***forehead', 'head***frown', 'head***hair', 'head***hair***sideburns',
              'head***jaw', 'head***moustache', 'head***mouth***inferior lip', 'head***mouth***oral comisure',
              'head***mouth***superior lip', 'head***mouth***teeth', 'head***neck', 'head***nose',
              'head***nose***ala of nose', 'head***nose***bridge', 'head***nose***nose tip', 'head***nose***nostril',
              'head***philtrum', 'head***temple', 'head***wrinkles']

car_12_class = ['background', 'car_body', 'head light', 'tail light', 'licence plate',
                'wind shield', 'wheel', 'door', 'handle' , 'wheelhub', 'window', 'mirror']
car_20_class = ['background', 'back_bumper', 'bumper', 'car_body', 'car_lights', 'door', 'fender','grilles','handles',
                'hoods', 'licensePlate', 'mirror','roof', 'running_boards', 'tailLight','tire', 'trunk_lids','wheelhub', 'window', 'windshield']


car_20_palette =[ 255,  255,  255, # 0 background
  238,  229,  102,# 1 back_bumper
  0, 0, 0,# 2 bumper
  124,  99 , 34, # 3 car
  193 , 127,  15,# 4 car_lights
  248  ,213 , 42, # 5 door
  220  ,147 , 77, # 6 fender
  99 , 83  , 3, # 7 grilles
  116 , 116 , 138,  # 8 handles
  200  ,226 , 37, # 9 hoods
  225 , 184 , 161, # 10 licensePlate
  142 , 172  ,248, # 11 mirror
  153 , 112 , 146, # 12 roof
  38  ,112 , 254, # 13 running_boards
  229 , 30  ,141, # 14 tailLight
  52 , 83  ,84, # 15 tire
  194 , 87 , 125, # 16 trunk_lids
  225,  96  ,18,  # 17 wheelhub
  31 , 102 , 211, # 18 window
  104 , 131 , 101# 19 windshield
         ]



face_palette = [  1.0000,  1.0000 , 1.0000,
              0.4420,  0.5100 , 0.4234,
              0.8562,  0.9537 , 0.3188,
              0.2405,  0.4699 , 0.9918,
              0.8434,  0.9329  ,0.7544,
              0.3748,  0.7917 , 0.3256,
              0.0190,  0.4943 , 0.3782,
              0.7461 , 0.0137 , 0.5684,
              0.1644,  0.2402 , 0.7324,
              0.0200 , 0.4379 , 0.4100,
              0.5853 , 0.8880 , 0.6137,
              0.7991 , 0.9132 , 0.9720,
              0.6816 , 0.6237  ,0.8562,
              0.9981 , 0.4692 , 0.3849,
              0.5351 , 0.8242 , 0.2731,
              0.1747 , 0.3626 , 0.8345,
              0.5323 , 0.6668 , 0.4922,
              0.2122 , 0.3483 , 0.4707,
              0.6844,  0.1238 , 0.1452,
              0.3882 , 0.4664 , 0.1003,
              0.2296,  0.0401 , 0.3030,
              0.5751 , 0.5467 , 0.9835,
              0.1308 , 0.9628,  0.0777,
              0.2849  ,0.1846 , 0.2625,
              0.9764 , 0.9420 , 0.6628,
              0.3893 , 0.4456 , 0.6433,
              0.8705 , 0.3957 , 0.0963,
              0.6117 , 0.9702 , 0.0247,
              0.3668 , 0.6694 , 0.3117,
              0.6451 , 0.7302,  0.9542,
              0.6171 , 0.1097,  0.9053,
              0.3377 , 0.4950,  0.7284,
              0.1655,  0.9254,  0.6557,
              0.9450  ,0.6721,  0.6162]

face_palette = [int(item * 255) for item in face_palette]





car_12_palette =[ 255,  255,  255, # 0 background
         124,  99 , 34, # 3 car
         193 , 127,  15,# 4 car_lights
         229 , 30  ,141, # 14 tailLight
        225 , 184 , 161, # 10 licensePlate
        104 , 131 , 101,# 19 windshield
        52 , 83  ,84, # 15 tire
        248  ,213 , 42, # 5 door
         116 , 116 , 138,  # 8 handles
           225,  96  ,18,  # 17 wheelhub
         31 , 102 , 211, # 18 window
         142 , 172  ,248, # 11 mirror
         ]



car_32_palette =[ 255,  255,  255,
  238,  229,  102,
  0, 0, 0,
  124,  99 , 34,
  193 , 127,  15,
  106,  177,  21,
  248  ,213 , 42,
  252 , 155,  83,
  220  ,147 , 77,
  99 , 83  , 3,
  116 , 116 , 138,
  63  ,182 , 24,
  200  ,226 , 37,
  225 , 184 , 161,
  233 ,  5  ,219,
  142 , 172  ,248,
  153 , 112 , 146,
  38  ,112 , 254,
  229 , 30  ,141,
  115  ,208 , 131,
  52 , 83  ,84,
  229 , 63 , 110,
  194 , 87 , 125,
  225,  96  ,18,
  73  ,139,  226,
  172 , 143 , 16,
  169 , 101 , 111,
  31 , 102 , 211,
  104 , 131 , 101,
  70  ,168  ,156,
  183 , 242 , 209,
  72  ,184 , 226]

bedroom_palette =[ 255,  255,  255,
  238,  229,  102,
  255, 72, 69,
  124,  99 , 34,
  193 , 127,  15,
  106,  177,  21,
  248  ,213 , 42,
  252 , 155,  83,
  220  ,147 , 77,
  99 , 83  , 3,
  116 , 116 , 138,
  63  ,182 , 24,
  200  ,226 , 37,
  225 , 184 , 161,
  233 ,  5  ,219,
  142 , 172  ,248,
  153 , 112 , 146,
  38  ,112 , 254,
  229 , 30  ,141,
   238, 229, 12,
   255, 72, 6,
   124, 9, 34,
   193, 17, 15,
   106, 17, 21,
   28, 213, 2,
   252, 155, 3,
   20, 147, 77,
   9, 83, 3,
   11, 16, 138,
   6, 12, 24,
   20, 22, 37,
   225, 14, 16,
   23, 5, 29,
   14, 12, 28,
   15, 11, 16,
   3, 12, 24,
   22, 3, 11
   ]

cat_palette = [255,  255,  255,
            220, 220, 0,
           190, 153, 153,
            250, 170, 30,
           220, 220, 0,
           107, 142, 35,
           102, 102, 156,
           152, 251, 152,
           119, 11, 32,
           244, 35, 232,
           220, 20, 60,
           52 , 83  ,84,
          194 , 87 , 125,
          225,  96  ,18,
          31 , 102 , 211,
          104 , 131 , 101
          ]

def trans_mask_stylegan_20classTo12(mask):
    final_mask = np.zeros(mask.shape)
    final_mask[(mask != 0)] = 1 # car
    final_mask[(mask == 4)] = 2 # head light
    final_mask[(mask == 14)] = 5 # tail light
    final_mask[(mask == 10)] = 3 # licence plate
    final_mask[ (mask == 19)] = 8 # wind shield
    final_mask[(mask == 15)] = 6 # wheel
    final_mask[(mask == 5)] = 9 # door
    final_mask[(mask == 8)] = 10 # handle
    final_mask[(mask == 17)] = 11 # wheelhub
    final_mask[(mask == 18)] = 7 # window
    final_mask[(mask == 11)] = 4 # mirror
    return final_mask


def trans_mask(mask):
    return mask

## Dataset

In [4]:
class ImageLabelDataset(Dataset):
    def __init__(
            self,
            img_path_list,
            label_path_list, trans,
            img_size=(128, 128),
    ):
        self.label_trans = trans
        self.img_path_list = img_path_list
        self.label_path_list = label_path_list
        self.img_size = img_size

    def __len__(self):
        return len(self.img_path_list)

    def __getitem__(self, index):
        im_path = self.img_path_list[index]
        lbl_path = self.label_path_list[index]
        im = Image.open(im_path)
        try:
            lbl = np.load(lbl_path)
        except:
            lbl = np.array(Image.open(lbl_path))
        if len(lbl.shape) == 3:
            lbl = lbl[:, :, 0]

        lbl = self.label_trans(lbl)
        lbl = Image.fromarray(lbl.astype('uint8'))
        im, lbl = self.transform(im, lbl)

        return im, lbl, im_path

    def transform(self, img, lbl):
        img = img.resize((self.img_size[0], self.img_size[1]))
        lbl = lbl.resize((self.img_size[0], self.img_size[1]), resample=Image.NEAREST)
        lbl = torch.from_numpy(np.array(lbl)).long()
        img = transforms.ToTensor()(img)
        return img, lbl


## Main Training Loop

In [5]:
def main(data_path, args, resume, max_data=0, uncertainty_portion=0):
    exp_path = args['exp_dir']

    base_path = os.path.join(exp_path, "deeplab_class_%d_checkpoint_%d_filter_out_%f" %(args['testing_data_number_class'],
                                                                                        int(max_data),
                                                                                        uncertainty_portion))
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    print("Model dir,", base_path)
    num_class = args['testing_data_number_class']


    dump_data = []
    all_pickle = glob.glob(data_path + '/*.pickle')

    used_image = []
    for p in all_pickle:
        with open(p, 'rb') as f:
            curr_dict = pickle.load(f)

        for dd in curr_dict:
            if not dd['image_name'] in used_image:
                used_image.append(dd['image_name'] )
                dump_data.append(dd)
    if max_data > 0:
        dump_data = dump_data[:max_data]
    stylegan_images = [data['image_name'] for data in dump_data]
    stylegan_labels = [data['image_label_name'] for data in dump_data]

    stylegan_images.sort()
    stylegan_labels.sort()
    if uncertainty_portion > 0:
        sort_by_uncertainty = sorted(dump_data, key=lambda k: k['uncertrainty_score'])
        filter_out_num = int(len(sort_by_uncertainty) * uncertainty_portion)
        sort_by_uncertainty = sort_by_uncertainty[30:-filter_out_num+ 30]
        out_idx = range(len(sort_by_uncertainty))
        stylegan_images = [sort_by_uncertainty[idx]['image_name'] for idx in out_idx]
        stylegan_labels = [sort_by_uncertainty[idx]['image_label_name'] for idx in out_idx]

    if args['number_class'] ==  args['testing_data_number_class']:
        trans_method = trans_mask
    else:
        # for testing on ADE-12 only. Since our generated images has 20 labels. Need to merge labels based on testing set.
        trans_method = trans_mask_stylegan_20classTo12
    assert  len(stylegan_images) == len(stylegan_labels)
    print( "Train data length,", str(len(stylegan_labels)))

    train_data = ImageLabelDataset(img_path_list=stylegan_images,
                              label_path_list=stylegan_labels, trans=trans_method,
                            img_size=(args['deeplab_res'], args['deeplab_res']))

    train_batch = 16

    train_data = DataLoader(train_data, batch_size=train_batch, shuffle=True, num_workers=16)
    classifier = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=False,
                                                                     num_classes=num_class, aux_loss=None)
    print(classifier.state_dict().keys())
    if resume != "":
        checkpoint = torch.load(resume)
        classifier.load_state_dict(checkpoint['model_state_dict'])

    if torch.cuda.is_available():
        classifier.cuda()
    classifier.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=0.001)

    resnet_transform = torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])


    for epoch in range(10):
        for i, da, in enumerate(train_data):
            if da[0].shape[0] != train_batch:
                continue
            if i % 10 == 0:
                gc.collect()

            classifier.train()

            optimizer.zero_grad()
            img, mask = da[0], da[1]

            if torch.cuda.is_available():
                img = img.cuda()
                mask = mask.cuda()

            input_img_tensor = []
            for b in range(img.size(0)):
                if img.size(1) == 4:
                    input_img_tensor.append(resnet_transform(img[b][:-1,:,:]))
                else:
                    input_img_tensor.append(resnet_transform(img[b]))

            input_img_tensor = torch.stack(input_img_tensor)

            y_pred = classifier(input_img_tensor)['out']
            loss = criterion(y_pred, mask)
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(epoch, 'epoch', 'iteration', i, 'loss', loss.item())

        model_path = os.path.join(base_path, 'deeplab_epoch_' + str(epoch) + '.pth')

        print('Save to:', model_path)
        torch.save({'model_state_dict': classifier.state_dict()},
                   model_path)

## Run

In [ ]:
!unzip model_dir_generated_dataset.zip

In [ ]:
args = {
    'data_path': 'model_dir/cat_16/samples',
    'exp': 'model_dir/cat_16/cat_16.json',
    'resume': '',
    'max_data': 0,
    'uncertainty_portion': 0
}

opts = json.load(open(args['exp'], 'r'))
print("Opt", opts)

path =opts['exp_dir']
if os.path.exists(path):
    pass
else:
    os.system('mkdir -p %s' % (path))
    print('Experiment folder created at: %s' % (path))

main(args['data_path'], opts, args['resume'], args['max_data'], args['uncertainty_portion'])



In [ ]:
!zip -r 'model_dir_downstream.zip' 'model_dir'

## Testing utils

In [6]:
def process_image(images):
    drange = [-1, 1]
    scale = 255 / (drange[1] - drange[0])
    images = images * scale + (0.5 - drange[0] * scale)

    images = images.astype(int)
    images[images > 255] = 255
    images[images < 0] = 0

    return images.astype(int)

def colorize_mask(mask, palette):
    # mask: numpy array of the mask

    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)
    return np.array(new_mask.convert('RGB'))

## Cross validation

In [7]:
def cross_validate(cp_path, args):
    if args['category'] == 'car':
        palette = car_20_palette
        ignore_index = -1
    elif args['category'] == 'face':
        palette = face_palette
        ignore_index = -1
    elif args['category'] == 'bedroom':
        palette = bedroom_palette
        ignore_index = 0
    elif args['category'] == 'cat':
        palette = cat_palette
        ignore_index = -1

    base_path = os.path.join(cp_path, "cross_validation")
    if not os.path.exists(base_path):
        os.mkdir(base_path)


    cps_all = glob.glob(cp_path + "/*")

    print(cps_all)

    cp_list = [data for data in cps_all if '.pth' in data and 'BEST' not in data]
    cp_list.sort()
    print('CP LIST')
    print(cp_list)

    ids = range(args['testing_data_number_class'])

    data_all = glob.glob(args['testing_path'] + "/*")
    images = [path for path in data_all if 'npy' not in path]
    labels = [path for path in data_all if 'npy' in path]
    images.sort()
    labels.sort()

    vis_data = ImageLabelDataset(img_path_list=images,
                                  label_path_list=labels, trans=trans_mask,
                                  img_size=(args['deeplab_res'], args['deeplab_res']))
    vis_data = DataLoader(vis_data, batch_size=1, shuffle=False, num_workers=0)
    vis = []
    for j, da, in enumerate(vis_data):
        img, mask = da[0], da[1]
        img = img.numpy()
        img = img * 255.

        img = np.transpose(img, (0, 2, 3, 1)).astype(np.uint8)

        mask = mask.numpy()

        curr_vis = np.concatenate( [img[0], colorize_mask(mask[0], palette)], 0 )
        if len(vis) < 50:

            vis.append(curr_vis)


    vis = np.concatenate(vis, 1)
    imageio.imwrite(   os.path.join(base_path, "testing.jpg"),
                      vis)

    fold_num =int( len(images) / 5)
    resnet_transform = torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])


    classifier = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=False,
                                                                     num_classes=args['testing_data_number_class'], aux_loss=None)

    cross_mIOU = []

    for i in range(5):
        print(i)
        val_image = images[fold_num * i: fold_num *i + fold_num]
        val_label = labels[fold_num * i: fold_num *i + fold_num]
        test_image = [img for img in images if img not in val_image]
        test_label =[label for label in labels if label not in val_label]
        print("Val Data length,", str(len(val_image)))
        print("Testing Data length,", str(len(test_image)))

        val_data = ImageLabelDataset(img_path_list=val_image,
                                      label_path_list=val_label, trans=trans_mask,
                                      img_size=(args['deeplab_res'], args['deeplab_res']))
        val_data = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

        test_data = ImageLabelDataset(img_path_list=test_image,
                                  label_path_list=test_label, trans=trans_mask,
                                img_size=(args['deeplab_res'], args['deeplab_res']))
        test_data = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=0)

        best_miou = 0
        best_val_miou = 0
        for resume in cp_list:
            print(resume)

            checkpoint = torch.load(resume)
            classifier.load_state_dict(checkpoint['model_state_dict'])


            classifier.cuda()
            classifier.eval()

            unions = {}
            intersections = {}
            for target_num in ids:
                unions[target_num] = 0
                intersections[target_num] = 0

            with torch.no_grad():
                for _, da, in enumerate(val_data):
                    print(_)

                    img, mask = da[0], da[1]

                    if img.size(1) == 4:
                        img = img[:, :-1, :, :]

                    img = img.cuda()
                    mask = mask.cuda()
                    input_img_tensor = []
                    for b in range(img.size(0)):
                        input_img_tensor.append(resnet_transform(img[b]))
                    input_img_tensor = torch.stack(input_img_tensor)

                    y_pred = classifier(input_img_tensor)['out']
                    y_pred = torch.log_softmax(y_pred, dim=1)
                    _, y_pred = torch.max(y_pred, dim=1)
                    y_pred = y_pred.cpu().detach().numpy()
                    mask = mask.cpu().detach().numpy()
                    bs = y_pred.shape[0]

                    curr_iou = []
                    if ignore_index > 0:
                        y_pred = y_pred * (mask != ignore_index)
                    for target_num in ids:
                        y_pred_tmp = (y_pred == target_num).astype(int)
                        mask_tmp = (mask == target_num).astype(int)

                        intersection = (y_pred_tmp & mask_tmp).sum()
                        union = (y_pred_tmp | mask_tmp).sum()

                        unions[target_num] += union
                        intersections[target_num] += intersection

                        if not union == 0:
                            curr_iou.append(intersection / union)
                mean_ious = []

                for target_num in ids:
                    mean_ious.append(intersections[target_num] / (1e-8 + unions[target_num]))
                mean_iou_val = np.array(mean_ious).mean()

                if mean_iou_val > best_val_miou:
                    best_val_miou = mean_iou_val
                    unions = {}
                    intersections = {}
                    for target_num in ids:
                        unions[target_num] = 0
                        intersections[target_num] = 0

                    with torch.no_grad():
                        testing_vis = []
                        for _, da, in enumerate(test_data):

                            img, mask = da[0], da[1]

                            if img.size(1) == 4:
                                img = img[:, :-1, :, :]

                            img = img.cuda()
                            mask = mask.cuda()
                            input_img_tensor = []
                            for b in range(img.size(0)):
                                input_img_tensor.append(resnet_transform(img[b]))
                            input_img_tensor = torch.stack(input_img_tensor)

                            y_pred = classifier(input_img_tensor)['out']
                            y_pred = torch.log_softmax(y_pred, dim=1)
                            _, y_pred = torch.max(y_pred, dim=1)
                            y_pred = y_pred.cpu().detach().numpy()
                            mask = mask.cpu().detach().numpy()

                            curr_iou = []
                            if ignore_index > 0:
                                y_pred = y_pred * (mask != ignore_index)
                            for target_num in ids:
                                y_pred_tmp = (y_pred == target_num).astype(int)
                                mask_tmp = (mask == target_num).astype(int)

                                intersection = (y_pred_tmp & mask_tmp).sum()
                                union = (y_pred_tmp | mask_tmp).sum()

                                unions[target_num] += union
                                intersections[target_num] += intersection

                                if not union == 0:
                                    curr_iou.append(intersection / union)


                            img = img.cpu().numpy()
                            img =  img * 255.
                            img = np.transpose(img, (0, 2, 3, 1)).astype(np.uint8)

                            curr_vis = np.concatenate([img[0], colorize_mask(y_pred[0], palette)], 0)
                            if len(testing_vis) < 50:
                                testing_vis.append(curr_vis)

                        testing_vis = np.concatenate(testing_vis, 1)
                        imageio.imwrite(os.path.join(base_path, "testing_round_%d.jpg" % i),
                                          testing_vis)

                        test_mean_ious = []

                        for target_num in ids:
                            test_mean_ious.append(intersections[target_num] / (1e-8 + unions[target_num]))
                        best_test_miou = np.array(test_mean_ious).mean()


                        print("Best IOU ,", str(best_test_miou), "CP: ", resume)

        cross_mIOU.append(best_test_miou)

    print(cross_mIOU)
    print(" cross validation mean:" , np.mean(cross_mIOU) )
    print(" cross validation std:", np.std(cross_mIOU))
    result = {"Cross validation mean": np.mean(cross_mIOU), "Cross validation std": np.std(cross_mIOU), "Cross validation":cross_mIOU }
    with open(os.path.join(cp_path, 'cross.json'), 'w') as f:
        json.dump(result, f)

## Testing

In [11]:
def test(cp_path, args, validation_number=50):
    if args['category'] == 'car':
        palette = car_20_palette
        if args['testing_data_number_class'] == 12:
            class_name = car_12_class
        elif args['testing_data_number_class'] == 20:
            class_name = car_20_class
    elif args['category'] == 'face':
        palette = face_palette
        class_name = face_class

    elif args['category'] == 'bedroom':
        palette = bedroom_palette
    
    elif args['category'] == 'cat':
        palette = cat_palette

    base_path = os.path.join(cp_path, "validation")
    if not os.path.exists(base_path):
        os.mkdir(base_path)


    cps_all = glob.glob(cp_path + "/*")

    cp_list = [data for data in cps_all if '.pth' in data and 'BEST' not in data]

    ids = range(args['testing_data_number_class'])

    data_all = glob.glob(args['testing_path'] + "/*")
    images = [path for path in data_all if 'npy' not in path]
    labels = [path for path in data_all if 'npy' in path]
    images.sort()
    labels.sort()

    vis_data = ImageLabelDataset(img_path_list=images,
                                  label_path_list=labels, trans=trans_mask,
                                  img_size=(args['deeplab_res'], args['deeplab_res']))
    vis_data = DataLoader(vis_data, batch_size=1, shuffle=False, num_workers=0)
    vis = []
    for j, da, in enumerate(vis_data):
        img, mask = da[0], da[1]
        img = img.numpy()
        img = img * 255.

        img = np.transpose(img, (0, 2, 3, 1)).astype(np.uint8)

        mask = mask.numpy()

        curr_vis = np.concatenate( [img[0], colorize_mask(mask[0], palette)], 0 )
        if len(vis) < 50:

            vis.append(curr_vis)

    vis = np.concatenate(vis, 1)
    imageio.imwrite(   os.path.join(base_path, "testing_gt.jpg"),
                      vis)


    resnet_transform = torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])


    classifier = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=False,
                                                                     num_classes=args['testing_data_number_class'], aux_loss=None)

    cross_mIOU = []
    if validation_number == 0:
        print("Report performance on the best checkpoint")
        val_image = images
        val_label = labels
        test_image = images
        test_label = labels
    else:
        val_image = images[:validation_number]
        val_label = labels[:validation_number]
        test_image = [img for img in images if img not in val_image]
        test_label =[label for label in labels if label not in val_label]
    print("Val Data length,", str(len(val_image)))
    print("Testing Data length,", str(len(test_image)))

    val_data = ImageLabelDataset(img_path_list=val_image,
                                  label_path_list=val_label, trans=trans_mask,
                                  img_size=(args['deeplab_res'], args['deeplab_res']))
    val_data = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=0)

    test_data = ImageLabelDataset(img_path_list=test_image,
                              label_path_list=test_label, trans=trans_mask,
                            img_size=(args['deeplab_res'], args['deeplab_res']))
    test_data = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=0)

    best_val_miou = 0

    cp_list.sort()
    for resume in cp_list:
        checkpoint = torch.load(resume)
        print(checkpoint['model_state_dict'])
        classifier.load_state_dict(checkpoint['model_state_dict'])


        classifier.cuda()
        classifier.eval()

        unions = {}
        intersections = {}
        for target_num in ids:
            unions[target_num] = 0
            intersections[target_num] = 0

        with torch.no_grad():
            for _, da, in enumerate(val_data):

                img, mask = da[0], da[1]

                if img.size(1) == 4:
                    img = img[:, :-1, :, :]

                img = img.cuda()
                mask = mask.cuda()
                input_img_tensor = []
                for b in range(img.size(0)):
                    input_img_tensor.append(resnet_transform(img[b]))
                input_img_tensor = torch.stack(input_img_tensor)

                y_pred = classifier(input_img_tensor)['out']
                y_pred = torch.log_softmax(y_pred, dim=1)
                _, y_pred = torch.max(y_pred, dim=1)
                y_pred = y_pred.cpu().detach().numpy()
                mask = mask.cpu().detach().numpy()
                bs = y_pred.shape[0]

                curr_iou = []

                for target_num in ids:
                    y_pred_tmp = (y_pred == target_num).astype(int)
                    mask_tmp = (mask == target_num).astype(int)

                    intersection = (y_pred_tmp & mask_tmp).sum()
                    union = (y_pred_tmp | mask_tmp).sum()

                    unions[target_num] += union
                    intersections[target_num] += intersection

                    if not union == 0:
                        curr_iou.append(intersection / union)
            mean_ious = []

            for target_num in ids:
                mean_ious.append(intersections[target_num] / (1e-8 + unions[target_num]))
            mean_iou_val = np.array(mean_ious).mean()

            if mean_iou_val > best_val_miou:
                best_val_miou = mean_iou_val
                unions = {}
                intersections = {}
                for target_num in ids:
                    unions[target_num] = 0
                    intersections[target_num] = 0

                with torch.no_grad():
                    testing_vis = []
                    for _, da, in enumerate(test_data):

                        img, mask = da[0], da[1]

                        if img.size(1) == 4:
                            img = img[:, :-1, :, :]

                        img = img.cuda()
                        mask = mask.cuda()
                        input_img_tensor = []
                        for b in range(img.size(0)):
                            input_img_tensor.append(resnet_transform(img[b]))
                        input_img_tensor = torch.stack(input_img_tensor)

                        y_pred = classifier(input_img_tensor)['out']
                        y_pred = torch.log_softmax(y_pred, dim=1)
                        _, y_pred = torch.max(y_pred, dim=1)
                        y_pred = y_pred.cpu().detach().numpy()
                        mask = mask.cpu().detach().numpy()

                        curr_iou = []

                        for target_num in ids:
                            y_pred_tmp = (y_pred == target_num).astype(int)
                            mask_tmp = (mask == target_num).astype(int)

                            intersection = (y_pred_tmp & mask_tmp).sum()
                            union = (y_pred_tmp | mask_tmp).sum()

                            unions[target_num] += union
                            intersections[target_num] += intersection

                            if not union == 0:
                                curr_iou.append(intersection / union)


                        img = img.cpu().numpy()
                        img =  img * 255.
                        img = np.transpose(img, (0, 2, 3, 1)).astype(np.uint8)

                        curr_vis = np.concatenate([img[0], colorize_mask(y_pred[0], palette)], 0)
                        if len(testing_vis) < 50:
                            testing_vis.append(curr_vis)

                    testing_vis = np.concatenate(testing_vis, 1)
                    imageio.imwrite(os.path.join(base_path, "testing.jpg"),
                                      testing_vis)

                    test_mean_ious = []

                    for j, target_num in enumerate(ids):
                        iou = intersections[target_num] / (1e-8 + unions[target_num])
                        print("IOU for ", target_num, iou)

                        test_mean_ious.append(iou)
                    best_test_miou = np.array(test_mean_ious).mean()
                    print("Best IOU ,", str(best_test_miou), "CP: ", resume)

    print(cross_mIOU)
    print("Validation mIOU:" ,best_val_miou)
    print("Testing mIOU:" , best_test_miou )

    result = {"Validation": best_val_miou, "Testing":best_test_miou}
    with open(os.path.join(cp_path, 'cross.json'), 'w') as f:
        json.dump(result, f)

In [ ]:
args = {
    'exp': 'cat_16.json',
    'resume': '.',
    'cross_validate': False,
    'validation_number': 0
}

opts = json.load(open(args['exp'], 'r'))
print("Opt", opts)

path =opts['exp_dir']
if os.path.exists(path):
    pass
else:
    os.system('mkdir -p %s' % (path))
    print('Experiment folder created at: %s' % (path))


if not args['cross_validate']:
    test(args['resume'], opts, args['validation_number'])
else:
    cross_validate(args['resume'], opts)


In [ ]:
!zip -r 'cv_baseline.zip' 'cross_validation'

In [ ]:
!zip -r 'val_baseline.zip' 'validation'